In [ ]:
# In [1]: Refactored Cell 1 — MB parsing + movie-guess prep

import re
import unicodedata
import pandas as pd
import os

# ---------- Config ----------
mb_path = r"D:\Capstone_Staging\data\soundtracks.tsv"

pd.set_option("display.max_rows", 60)
pd.set_option("display.max_colwidth", 120)

MIN_WORDS_IN_GUESS = 2
YEAR_MIN, YEAR_MAX = 1930, 2025
MAX_TOKENS = 8
USE_YEAR_FALLBACK = True
INCLUDE_MINISERIES = True

out_dir = r"D:\Capstone_Staging\debug"
os.makedirs(out_dir, exist_ok=True)
out_path = os.path.join(out_dir, "mb_probe_hits.csv")
probe_hits.to_csv(out_path, index=False)

# ---------- Load ----------
mb = pd.read_csv(
    mb_path,
    sep="\t",
    header=None,
    usecols=[2, 18],  # title, created_date
    names=["MB_title", "MB_created_date"],
    dtype=str,
)
mb["MB_year_created"] = pd.to_datetime(mb["MB_created_date"], errors="coerce").dt.year

# ---------- Cleaners ----------
OST_PATTERNS = [
    r"\boriginal\s+motion\s+picture\s+soundtrack\b",
    r"\bmotion\s+picture\s+soundtrack\b",
    r"\bmusic\s+from\s+and\s+inspired\s+by\b",
    r"\bmusic\s+from\s+the\s+(?:motion\s+)?picture\b",
    r"\boriginal\s+soundtrack\b",
    r"\boriginal\s+score\b",
    r"\b(?:the\s+)?soundtrack(?:\s+album)?\b",
    r"\bscore\b",
    r"\bost\b",
    r"\bdeluxe\s+edition\b",
    r"\bexpanded\s+edition\b",
    r"\bthe\s+motion\s+picture\b",
    r"\bthe\s+series\b" if INCLUDE_MINISERIES else r"$a"
]

def nfkc(s: str) -> str:
    return unicodedata.normalize("NFKC", str(s or "")).strip()

def normalize_ws(s: str) -> str:
    return re.sub(r"\s+", " ", s).strip()

def clean_title(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = nfkc(s).lower()
    s = re.sub(r"\([^)]*\)", " ", s)
    s = re.sub(r"\[[^\]]*\]", " ", s)
    for p in OST_PATTERNS:
        s = re.sub(p, " ", s, flags=re.IGNORECASE)
    s = re.sub(r"[^0-9a-z\s\-–:’'&.,!?()]", " ", s)
    return normalize_ws(s)

_LATINISH = re.compile(r"^[a-z0-9\s\-–:’'&.,!?()]+$", re.IGNORECASE)
def is_latinish(s: str) -> bool:
    return bool(_LATINISH.match(nfkc(str(s))))

def year_from_title(s: str):
    m = re.search(r"(?:19|20)\d{2}", str(s))
    return int(m.group()) if m else None

_SPLIT = re.compile(r"\s*[-–:]\s*")

def guess_movie_from_album(title: str) -> str:
    cleaned = clean_title(title)
    if len(re.sub(r"[^a-z]", "", cleaned)) < 3:
        return ""
    head = _SPLIT.split(cleaned)[0]
    tokens = head.split()
    if len(tokens) > MAX_TOKENS:
        tokens = tokens[:MAX_TOKENS]
    return " ".join(tokens)

# ---------- Build working frame ----------
mb["MB_movie_guess"] = mb["MB_title"].astype(str).map(guess_movie_from_album)
mb["MB_year_from_title"] = mb["MB_title"].map(year_from_title)
mb["MB_year"] = mb["MB_year_from_title"].fillna(mb["MB_year_created"]).astype("Int64")

mb = mb[mb["MB_movie_guess"].str.len() >= 3]
mb = mb[mb["MB_movie_guess"].map(is_latinish)]

# ---------- Filters ----------
SOUNDTRACK_CUES = re.compile(
    r"(?i)\b(?:soundtrack|original(?:\s+motion\s+picture)?\s+soundtrack|original\s+score|"
    r"motion\s+picture|film\s+score|bande\s+originale|"
    r"music\s+from(?:\s+and\s+inspired\s+by)?\s+the\s+(?:motion\s+)?picture|"
    r"score|ost|musical\s+experience|complete\s+musical\s+experience)\b"
)

COMPILATION_NOISE = re.compile(
    r"(?i)\b(?:best\s+of|greatest\s+hits|now\s+that.?s\s+what\s+i\s+call\s+music|"
    r"volume\s*\d+|vol\.?\s*\d+|disc\s*\d+|cd\s*\d+|annual|sampler|series|"
    r"presents|mix|club|radio|remix|anthology|collection|compilation|"
    r"unmixed|dj\s+mix|bootleg)\b"
)

FRANCHISE_NOISE = re.compile(
    r"(?i)\b(?:special\s+edition|expanded|anniversary|remastered)\b"
)

cue_mask  = mb["MB_title"].str.contains(SOUNDTRACK_CUES, regex=True, na=False)
year_mask = mb["MB_title"].str.contains(r"(?:19|20)\d{2}", regex=True, na=False) if USE_YEAR_FALLBACK else pd.Series(False, index=mb.index)

# stricter movie-shape heuristic
GENERIC_BAD_WORDS = {"friends", "classics", "glory", "hits", "love", "jazz", "mix"}
def movie_shape(s: str) -> bool:
    words = s.split()
    if len(words) < 2 or len(words) > 3:
        return False
    if any(w.isdigit() for w in words):
        return False
    if not words[0][0].isupper():
        return False
    if any(w.lower() in GENERIC_BAD_WORDS for w in words):
        return False
    return True

shape_mask = mb["MB_movie_guess"].map(movie_shape)

# Admission = cues OR year OR shape
admit_mask = cue_mask | year_mask | shape_mask

mb = mb[admit_mask].copy()
mb = mb[~mb["MB_title"].str.contains(COMPILATION_NOISE, regex=True, na=False)]
mb = mb[~mb["MB_title"].str.contains(FRANCHISE_NOISE, regex=True, na=False)]

# Guards
mb = mb[mb["MB_movie_guess"].str.count(r"\b\w+\b") >= MIN_WORDS_IN_GUESS]
mb = mb[mb["MB_year"].isna() | ((mb["MB_year"] >= YEAR_MIN) & (mb["MB_year"] <= YEAR_MAX))]
mb = mb.drop_duplicates(subset=["MB_movie_guess", "MB_year"]).reset_index(drop=True)

# ---------- Preview + Diagnostics ----------
print("After refined filtering:")
print(mb.head(12).to_string(index=False))
print("\nCounts — rows:", len(mb), "| distinct guesses:", mb["MB_movie_guess"].nunique())

# Probes
probes = ["la la land", "whiplash", "amadeus", "interstellar"]
for probe in probes:
    hits = mb[mb["MB_movie_guess"].str.contains(probe, regex=False)]
    if hits.empty:
        print(f"{probe} → 0  (MISSING in MB titles)")
    else:
        print(f"{probe} → {len(hits)}")

# Admission path breakdown
cue_mask_final   = mb["MB_title"].str.contains(SOUNDTRACK_CUES, regex=True, na=False)
year_mask_final  = mb["MB_title"].str.contains(r"(?:19|20)\d{2}", regex=True, na=False)
shape_mask_final = mb["MB_movie_guess"].map(movie_shape)

def classify_row(cue, year, shape):
    if cue: return "cue"
    elif year: return "year"
    elif shape: return "shape"
    else: return "other"

admit_paths = pd.DataFrame({
    "cue": cue_mask_final,
    "year": year_mask_final,
    "shape": shape_mask_final
})
admit_paths["admission"] = admit_paths.apply(
    lambda row: classify_row(row["cue"], row["year"], row["shape"]), axis=1
)
path_counts = admit_paths["admission"].value_counts().to_dict()
print("\nAdmission path breakdown:")
for k, v in path_counts.items():
    print(f"  {k:5s}: {v}")

# Export probe hits for inspection
probe_hits = pd.DataFrame()
for probe in probes:
    hits = mb[mb["MB_movie_guess"].str.contains(probe, regex=False)]
    if not hits.empty:
        hits = hits.copy()  # avoid SettingWithCopyWarning
        hits.loc[:, "probe"] = probe
        probe_hits = pd.concat([probe_hits, hits])

if not probe_hits.empty:
    out_dir = r"D:\Capstone_Staging\debug"
    os.makedirs(out_dir, exist_ok=True)
    out_path = os.path.join(out_dir, "mb_probe_hits.csv")
    probe_hits.to_csv(out_path, index=False)
    print(f"\nExported probe hits to {out_path} ({len(probe_hits)} rows).")

# --- Diagnostics: why Interstellar dropped ---
diag_probe = "interstellar"

# Search in raw titles (before cleaning)
mb_raw = pd.read_csv(
    mb_path,
    sep="\t",
    header=None,
    usecols=[2, 18],  # title, created_date
    names=["MB_title", "MB_created_date"],
    dtype=str,
)

raw_hits = mb_raw[mb_raw["MB_title"].str.contains(diag_probe, case=False, na=False)]
print(f"\n=== Raw hits for '{diag_probe}' in MB_title: {len(raw_hits)} rows ===")
if not raw_hits.empty:
    print(raw_hits.head(20).to_string(index=False))

# Search in cleaned/filtered DataFrame before final admission
clean_hits = mb[mb["MB_movie_guess"].str.contains(diag_probe, case=False, na=False)]
print(f"\n=== Survived filtering for '{diag_probe}': {len(clean_hits)} rows ===")
if not clean_hits.empty:
    print(clean_hits.head(20).to_string(index=False))



In [ ]:
# --- Cell 2: MB ↔ TMDb fuzzy matching (batch mode) ---

import os
import requests
import pandas as pd
from rapidfuzz import fuzz

TMDB_API_KEY = os.getenv("TMDB_API_KEY")
if not TMDB_API_KEY:
    raise RuntimeError("TMDB_API_KEY not found in environment variables.")

# --- Helper: TMDb search ---
def tmdb_search_movie(query, year=None, max_pages=2):
    results = []
    base_url = "https://api.themoviedb.org/3/search/movie"
    params = {
        "api_key": TMDB_API_KEY,
        "query": query,
        "include_adult": "false",
    }
    if year:
        params["year"] = int(year)

    for page in range(1, max_pages + 1):
        params["page"] = page
        r = requests.get(base_url, params=params)
        if r.status_code != 200:
            break
        data = r.json()
        results.extend(data.get("results", []))
        if page >= data.get("total_pages", 1):
            break
    return results

# --- Clean OST boilerplate for query ---
OST_NOISE = [
    "original motion picture soundtrack",
    "motion picture soundtrack",
    "music from and inspired by",
    "soundtrack",
    "score",
    "ost",
    "deluxe edition",
    "expanded edition",
    "music from the motion picture",
    "original soundtrack"
]

def clean_for_tmdb_query(title: str) -> str:
    t = title.lower()
    for noise in OST_NOISE:
        t = t.replace(noise, "")
    return " ".join(t.split()).strip()

# --- Fuzzy match wrapper ---
def best_tmdb_match(mb_title, mb_year=None, year_bias=2):
    if not mb_title or not isinstance(mb_title, str):
        return None

    query_title = clean_for_tmdb_query(mb_title)
    tmdb_results = tmdb_search_movie(query_title, year=None, max_pages=2)
    if not tmdb_results:
        return None

    scored = []
    for r in tmdb_results:
        title = r.get("title") or ""
        release_date = r.get("release_date") or ""
        release_year = int(release_date[:4]) if release_date else None

        score = fuzz.ratio(query_title.lower(), title.lower())

        # Year bonus (only positive bias, no harsh penalty)
        if mb_year and release_year and abs(int(mb_year) - release_year) <= year_bias:
            score += 5

        scored.append((score, r))

    if not scored:
        return None
    best_score, best = max(scored, key=lambda x: x[0])

    return {
        "TMDb_id": best.get("id"),
        "TMDb_title": best.get("title"),
        "TMDb_year": best.get("release_date", "")[:4],
        "TMDb_popularity": best.get("popularity"),
        "TMDb_genres_ids": best.get("genre_ids"),
        "match_score": best_score,
    }

# --- Batch process all MB rows ---
batch_matches = []
for idx, row in mb.iterrows():
    m = best_tmdb_match(row["MB_movie_guess"], row["MB_year"])
    if m:
        out = {
            "MB_title": row["MB_title"],
            "MB_year": row["MB_year"],
            "MB_movie_guess": row["MB_movie_guess"],
        }
        out.update(m)
        batch_matches.append(out)
    if idx % 500 == 0 and idx > 0:
        print(f"Processed {idx} rows...")

matches_df = pd.DataFrame(batch_matches)
print("Batch matching complete.")
print("Matches:", len(matches_df))

# Save interim results
out_path = r"D:\Capstone_Staging\debug\mb_tmdb_matches.csv"
matches_df.to_csv(out_path, index=False)
print(f"Exported matches to {out_path}")


In [ ]:
# --- Cell 3: Genre mapping + scatterplot ---

import matplotlib.pyplot as plt

# Static TMDb genre map (as of 2024, can be refreshed via API if needed)
TMDB_GENRE_MAP = {
    28: "Action",
    12: "Adventure",
    16: "Animation",
    35: "Comedy",
    80: "Crime",
    99: "Documentary",
    18: "Drama",
    10751: "Family",
    14: "Fantasy",
    36: "History",
    27: "Horror",
    10402: "Music",
    9648: "Mystery",
    10749: "Romance",
    878: "Science Fiction",
    10770: "TV Movie",
    53: "Thriller",
    10752: "War",
    37: "Western"
}

# Work on a filtered copy
df = matches_df.copy()

# Only keep strong matches
df = df[df["match_score"] >= 90]

# Map genre_ids to names
def map_genres(ids):
    if not isinstance(ids, list): return []
    return [TMDB_GENRE_MAP.get(i, str(i)) for i in ids]

df["genres"] = df["TMDb_genres_ids"].apply(map_genres)
df["genre_diversity"] = df["genres"].apply(lambda g: len(set(g)))

print("Sample mapped genres:")
print(df[["TMDb_title", "TMDb_year", "genres", "genre_diversity"]].head(10).to_string(index=False))

# --- Scatterplot: Popularity vs Genre Diversity ---
plt.figure(figsize=(8,6))
plt.scatter(df["TMDb_popularity"], df["genre_diversity"], alpha=0.6)
plt.xlabel("TMDb Popularity")
plt.ylabel("Genre Diversity (# unique genres)")
plt.title("Movie Popularity vs Genre Diversity (Matches ≥ 90)")
plt.grid(True, alpha=0.3)
plt.show()
